In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

import os
import numpy as np
import pandas as pd
# Custom utils
from utils.simulator.simulator import MCSimulation
# Tf imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

2023-03-19 11:58:48.423721: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def custom_derivative_model(x, model, multioutput = True, mode = 'centered'):
    h = 1e-1
    # Size x
    x_dim, y_dim = x.shape
    # Gradient vector
    if multioutput:
        gradient = np.zeros((x_dim, y_dim, y_dim))
    else:
        gradient = np.zeros((x_dim, y_dim, 1))
    for i in range(y_dim):
        # Vector for partial derivative estimation
        offset_tensor = np.zeros((x_dim, y_dim))
        offset_tensor[:, i] = h
        offset_tensor = tf.convert_to_tensor(offset_tensor,
                                             dtype = tf.float32)
        # Constantes:
        denominator = h
        if mode == 'progressive':
            numerator = model(
                tf.math.add(x, offset_tensor)
            ) - model(
                x
            ) 
        elif mode == 'regressive':
            numerator = model(
                x
            ) - model(
                tf.math.subtract(x, offset_tensor)
            )
        elif mode == 'centered':
            numerator = tf.math.subtract(
                model(
                    tf.math.add(x, offset_tensor)
                ), model(
                    tf.math.subtract(x, offset_tensor)
                )
            )
        denominator = 2 * h
        gradient [:, i, :] = numerator / denominator
    gradient = tf.convert_to_tensor(gradient,
                                        dtype = tf.float32)
    return gradient

In [47]:
input_layer = keras.Input(shape = (10,), name='input_nn')
output_layer = layers.Dense(
    units = 10, 
    activation = 'relu', 
    name = 'first_dense'
)(input_layer)
custom_model = keras.Model(
    inputs=[input_layer],
    outputs=[output_layer],
    name = 'test_model'
)

In [48]:
import time
# Domain -5:5
x = tf.convert_to_tensor(
    np.array(range(0,30)).reshape((3,10)),
    dtype = tf.float32    
)
# Test 
tf_start = time.time()
xs = tf.Variable(x, trainable = True, name = 'x')
with tf.GradientTape() as tape, tf.GradientTape() as tape_2:
    tape.watch(xs)
    tape_2.watch(xs)
    y = custom_model(xs)
tf_end = time.time()
print(f'Execution time: {tf_end - tf_start}')
# This represents dV/dX
grads = tape.gradient(y, {
    'x':xs
})
jacobian = tape_2.jacobian(y, {
    'x':xs
})

Execution time: 0.008378267288208008


In [49]:
custom_start = time.time()
custom_grads = custom_derivative_model(x, custom_model)
custom_end = time.time()
print(f'Execution time custom: {custom_end - custom_start}')

Execution time custom: 0.013373851776123047


In [50]:
jacobian_reshaped = jacobian['x']

In [51]:
custom_grads, tf.linalg.diag_part(custom_grads), jacobian_reshaped

(<tf.Tensor: shape=(3, 10, 10), dtype=float32, numpy=
 array([[[ 0.18664002,  0.        ,  0.        ,  0.        ,
           0.        ,  0.27842522,  0.06401777, -0.15793085,
           0.        ,  0.39049387],
         [-0.50091624,  0.        ,  0.        ,  0.        ,
           0.        , -0.1376152 , -0.23480654,  0.33662677,
           0.        ,  0.19052982],
         [-0.10197639,  0.        ,  0.        ,  0.        ,
           0.        , -0.3005767 , -0.3016472 , -0.09532213,
           0.        ,  0.0545454 ],
         [-0.04619718,  0.        ,  0.        ,  0.        ,
           0.        ,  0.33932447, -0.53911686,  0.23003936,
           0.        , -0.06365299],
         [ 0.1286745 ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.22497892,  0.42009115,  0.16330004,
           0.        , -0.11050224],
         [ 0.06087422,  0.        ,  0.        ,  0.        ,
           0.        ,  0.36040545,  0.2978444 , -0.43895006,
           0.    